In [1]:
from pyspark.sql import SparkSession
import re
from operator import add
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

In [2]:
rdd = sc.textFile('epa-http.txt')
url_pat = re.compile('(([0-9]|[1-9][0-9]|1[0-9]{2}|2[0-4][0-9]|25[0-5])\.){3}([0-9]|[1-9][0-9]|1[0-9]{2}|2[0-4][0-9]|25[0-5]).*')
rdd = rdd.filter(lambda x: url_pat.match(x) and x[-1] != '-').map(lambda x: x.split(' '))
rdd = rdd.map(lambda x: [x[0], x[1], x[-1]])

In [3]:
def process_map(x):
    num_list = x[0].split('.')
    num_list = num_list[0: -1]
    subnet = '.'.join(num_list)
    subnet += '.*'
    time = x[1][1: -1]
    time_list = time.split(':')
    sec = ((int(time_list[0]) - 29) * 86400 + int(time_list[1]) * 3600 + \
           int(time_list[2]) * 60 + int(time_list[3]) - (23 * 3600 + 53 * 60 + 25))
    return [(subnet, sec / 3600), int(x[-1])]

In [4]:
rdd_process = rdd.map(process_map)

In [5]:
bytes = rdd_process.map(lambda x: [x[0][0], int(x[-1])])
bytes = bytes.reduceByKey(add)
max_bytes = bytes.sortBy(lambda x: x[1], ascending = False)
res1 = bytes.collect()
res2 = max_bytes.take(5)

In [6]:
count = rdd_process.reduceByKey(add)
res3 = count.collect()

In [7]:
for k, v in res1:
    print v, 'bytes are served to subnet:', k
print '\nThe top 5 subnets served are:'
for k, v in res2:
    print v, 'bytes are served to', k
print '\n'
for k, v in res3:
    print v, 'bytes are served to', k[0], 'in the hour period', k[1] + 1

12588 bytes are served to subnet: 131.170.154.*
86425 bytes are served to subnet: 130.101.13.*
2235 bytes are served to subnet: 137.132.52.*
80786 bytes are served to subnet: 192.246.73.*
147822 bytes are served to subnet: 150.200.42.*
20658 bytes are served to subnet: 202.46.240.*
271513 bytes are served to subnet: 164.155.1.*
29844 bytes are served to subnet: 204.62.25.*
204373 bytes are served to subnet: 204.248.203.*
14449 bytes are served to subnet: 137.159.177.*
0 bytes are served to subnet: 200.12.140.*
14416 bytes are served to subnet: 134.136.91.*
12306 bytes are served to subnet: 129.237.223.*
6255 bytes are served to subnet: 149.168.105.*
98546 bytes are served to subnet: 129.81.186.*
310715 bytes are served to subnet: 192.48.125.*
2873 bytes are served to subnet: 163.234.52.*
8618 bytes are served to subnet: 137.241.180.*
5388 bytes are served to subnet: 128.180.28.*
8361 bytes are served to subnet: 204.238.43.*
81723 bytes are served to subnet: 162.43.21.*
16131 bytes are 

47666 bytes are served to subnet: 199.222.162.*
444230 bytes are served to subnet: 141.161.18.*
398729 bytes are served to subnet: 142.28.2.*
15663 bytes are served to subnet: 134.243.230.*
60244 bytes are served to subnet: 205.133.189.*
11747 bytes are served to subnet: 192.107.65.*
24398 bytes are served to subnet: 128.196.182.*
116718 bytes are served to subnet: 206.27.204.*
104910 bytes are served to subnet: 204.177.210.*
13603 bytes are served to subnet: 130.54.47.*
18257 bytes are served to subnet: 158.111.97.*
259921 bytes are served to subnet: 150.216.15.*
8280 bytes are served to subnet: 146.114.111.*
19910 bytes are served to subnet: 155.75.107.*
75178 bytes are served to subnet: 199.73.129.*
24166 bytes are served to subnet: 134.186.191.*
15699 bytes are served to subnet: 148.149.108.*
82766 bytes are served to subnet: 155.82.161.*
134858 bytes are served to subnet: 139.121.79.*
41943 bytes are served to subnet: 204.165.200.*
88485 bytes are served to subnet: 193.145.151.*
2

103106 bytes are served to 141.233.130.* in the hour period 16
11808 bytes are served to 155.81.101.* in the hour period 23
26174 bytes are served to 204.176.47.* in the hour period 24
4889 bytes are served to 147.154.93.* in the hour period 11
0 bytes are served to 38.12.1.* in the hour period 23
18323 bytes are served to 192.217.118.* in the hour period 14
6236 bytes are served to 138.13.122.* in the hour period 14
393035 bytes are served to 160.69.35.* in the hour period 20
11746 bytes are served to 165.235.39.* in the hour period 17
14450 bytes are served to 164.156.42.* in the hour period 12
6347 bytes are served to 157.145.93.* in the hour period 14
33221 bytes are served to 198.234.90.* in the hour period 15
17074 bytes are served to 128.159.144.* in the hour period 9
0 bytes are served to 200.12.140.* in the hour period 17
35792 bytes are served to 129.228.127.* in the hour period 14
672345 bytes are served to 198.106.169.* in the hour period 17
4449 bytes are served to 134.116

63661 bytes are served to 192.146.118.* in the hour period 5
19645 bytes are served to 200.10.224.* in the hour period 12
34095 bytes are served to 144.39.248.* in the hour period 14
81936 bytes are served to 163.234.140.* in the hour period 10
23494 bytes are served to 141.243.1.* in the hour period 6
10901 bytes are served to 204.4.34.* in the hour period 16
2730 bytes are served to 199.249.179.* in the hour period 17
9785 bytes are served to 155.206.3.* in the hour period 10
18280 bytes are served to 194.166.2.* in the hour period 21
24866 bytes are served to 140.96.17.* in the hour period 10
8618 bytes are served to 137.241.180.* in the hour period 13
47538 bytes are served to 198.105.194.* in the hour period 14
86658 bytes are served to 132.170.199.* in the hour period 19
4888 bytes are served to 134.67.75.* in the hour period 15
98546 bytes are served to 129.81.186.* in the hour period 15
8718 bytes are served to 155.76.34.* in the hour period 11
1356760 bytes are served to 146.1

10297 bytes are served to 140.112.34.* in the hour period 8
178862 bytes are served to 155.84.91.* in the hour period 10
161147 bytes are served to 134.186.196.* in the hour period 18
2624 bytes are served to 146.229.1.* in the hour period 10
23026 bytes are served to 130.207.94.* in the hour period 16
8727 bytes are served to 131.182.170.* in the hour period 15
3785626 bytes are served to 203.251.228.* in the hour period 4
7811 bytes are served to 140.112.68.* in the hour period 1
26405 bytes are served to 204.242.57.* in the hour period 10
93491 bytes are served to 156.63.241.* in the hour period 16
68101 bytes are served to 155.79.153.* in the hour period 14
11746 bytes are served to 128.150.157.* in the hour period 13
8934 bytes are served to 204.62.245.* in the hour period 15
6833 bytes are served to 192.137.113.* in the hour period 13
26138 bytes are served to 142.139.220.* in the hour period 17
11747 bytes are served to 204.157.204.* in the hour period 10
7574 bytes are served t